In [1]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import oct2py
from oct2py import octave

import math
from scipy.spatial import distance
import scipy.signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import metrics

In [2]:
def iris(Image):
    Enhanced=[]
    oc = oct2py.Oct2Py()
    oc.eval('pkg load image')
    for idx in range(len(Image)):
        filt=3
        par=0.0
        while True:
            imgi=Image[idx].copy()
            blur=imgi
            oc.push('I', blur)
            oc.push('par',par)
            oc.eval("I=uint8(I);");
            oc.eval("seg=1-im2bw(I,graythresh(I,'maxentropy')-par);")
            seg=oc.pull('seg')
            seg=seg.astype('uint8')
            blur=255*seg    
            blur=cv2.blur(blur,(3,3))
            blur=cv2.medianBlur(blur,9)
            for i in range(40):
                blur = cv2.bilateralFilter(blur, 4,75,75)
            seg=blur
            img = cv2.blur(seg,(1,1))
            circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,150,param1=50,param2=30,minRadius=0,maxRadius=0)
            if circles is not None:
                break
            else:
                filt=filt+2
                if filt>=15:
                    filt=3
                    par+=0.1
        circles = np.uint16(np.around(circles))
        for j in circles[0,:]:
            center_x = j[0]
            center_y = j[1]
            radius_pupil=int(j[2])
            
            iris_radius = 53
        
        nsamples = 360
        samples = np.linspace(0,2*np.pi, nsamples)[:-1]
        polar = np.zeros((iris_radius, nsamples))
    
        for r in range(iris_radius):
            for theta in samples:
                #get x and y for values on inner boundary
                x = (r+radius_pupil)*np.cos(theta)+center_x
                y = (r+radius_pupil)*np.sin(theta)+center_y
                x=int(x)
                y=int(y)
                try:
                #convert coordinates
                    polar[r][int((theta*nsamples)/(2*np.pi))] = imgi[y][x]
                except IndexError: #ignores values which lie out of bounds
                    pass
                continue
        polar=polar.astype(np.uint8)
        polar=cv2.resize(polar,(512,64))
        enhanced=cv2.equalizeHist(polar)
        Enhanced.append(enhanced)
        #cv2.imshow("xy",enhanced)
        #cv2.waitKey()
    return Enhanced

In [3]:
def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 8x8 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            sfilter[i,j]=gabor((-4+j),(-4+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(6):
            for j in range(64):
                #Run 8 by 8 filtered block iteratively over the entire image
                start_height = i*8
                end_height = start_height+8
                start_wid = j*8
                end_wid = start_wid+8
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[]
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector #each feature vector has a dimension of 1536


In [4]:
with open('name_L.pkl', 'rb') as f:
    name_L = pickle.load(f)
    
with open('img_L.pkl', 'rb') as f:
    img_L = pickle.load(f)

In [5]:
length=[[len(img[0]),len(img[1])] for img in img_L]
length=np.asarray(length)
indices=np.transpose(np.nonzero(length>=7))

In [6]:
train_names=[]
test_names=[]
train_images=[]
test_images=[]
for i in range(len(indices)):
    train_names.append(name_L[indices[i][0]][indices[i][1]][:4])
    test_names.append(name_L[indices[i][0]][indices[i][1]][4:])
    train_images.append(img_L[indices[i][0]][indices[i][1]][:4])
    test_images.append(img_L[indices[i][0]][indices[i][1]][4:])

In [7]:
with open('sklearn_lda.pkl', 'rb') as f:
    model = pickle.load(f)

H:\Heavy_softwares\Anaconda3\envs\venv2\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearDiscriminantAnalysis from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
Image=[train_images[104][0]]#some list
Enhanced=iris(Image)
feature_vector=FeatureExtraction(Enhanced)
A=model.transform(feature_vector)

In [15]:
Image=[test_images[105][2]]#some list
Enhanced=iris(Image)
feature_vector=FeatureExtraction(Enhanced)
B=model.transform(feature_vector)

In [16]:
cos_sim=(np.dot(A,B.T))/(np.sqrt(np.dot(A,A.T))*np.sqrt(np.dot(B,B.T)))
print(cos_sim)

[[0.13608274]]


In [17]:
thresh=0.4
if cos_sim>=thresh:
    print("Authenticated.")
else:
    print("Not authenticated.")

Not authenticated.
